In [1]:
from dash import Dash, html, dcc
from AnalysisTools import visual
from LoaderTools.libs import LibsLoader
from AnalysisTools.analytics import AnalyticsToolkit

import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import plotly.express as px
import numpy as np

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 1
%autoreload 2

In [2]:
fname = r"E:/Data/Data_LIBS/ForHolo/wrench_map"
data_handler = LibsLoader(fname)
data_handler.load_dataset(baseline_corrected=True)
data_handler.normalize_to_sum()
data_handler.automatic_feature_extraction(fft_features=0, intens_features=20, sigma=1)

In [3]:
analytics = AnalyticsToolkit()
labels = analytics.clustering(model = 'kmeans', 
                               n_clusters = 4, 
                               feature_cube = data_handler.features, 
                               scaler = 'minmax',
                               random_state = 10)

In [234]:
def create_spectral_image_plot(data_handler):
    wv = 670.76
    slice_data = data_handler.dataset[:, :, data_handler.wavelength_to_index(wv)]
    
    # Create the figure without colorbar and title
    fig = px.imshow(
        slice_data,
        color_continuous_scale='turbo',
        labels={'color': 'Intensity'}
    )

    # Adjust layout to match the style
    fig.update_layout(
        width=365,
        height=355,
        title={
            'text': "Spatial Distribuition",
            'x': 0.45,
            'y': 0.9,
            'xanchor': 'center',
            'font': {'size': 22}
        },
        margin=dict(l=0, r=0, t=0, b=0),
        xaxis=dict(
            title={
                'text': "x (mm)",
                'font': {'size': 18},
                'standoff': 0,
            },
            range=[0, slice_data.shape[1] - 1],
            tickvals=[1, slice_data.shape[1] - 1],
            ticktext=[0, slice_data.shape[1]],
            tickfont=dict(size=16, color='black'),
            linewidth=2,
            linecolor='black',
            mirror=True
        ),
        yaxis=dict(
            title={
                'text': "y (mm)",
                'font': {'size': 18},
                'standoff': 0,
            },
            range=[0, slice_data.shape[0] - 1],
            tickvals=[1, slice_data.shape[0] - 1],
            ticktext=[slice_data.shape[0], 0],
            tickfont=dict(size=16, color='black'),
            linewidth=2,
            linecolor='black',
            mirror=True
        ),
        plot_bgcolor='rgba(0,0,0,0)',  # Transparent background
        paper_bgcolor='rgba(0,0,0,0)',  # Transparent background
        font=dict(color='black'),
        coloraxis_colorbar=dict(
            thickness=10,
            len=0.64,
            y=0.5,
            x=1.05,
            yanchor='middle',
            ticks='outside',
            tickvals=[slice_data.min(), slice_data.max()],
            ticktext=[f"{slice_data.min():.3f}", f"{slice_data.max():.3f}"],
            tickfont=dict(size=14, color='black'),
            tickcolor='black',
            title={'text': ''}  # Empty title
        )
    )
    return fig

create_spectral_image_plot(data_handler)

In [251]:
def create_average_spectrum_plot(data_handler):
    max_spectrum = np.max(data_handler.dataset, axis=(0, 1))
    
    fig = px.line(
        x=data_handler.wavelengths,
        y=max_spectrum,
        color_discrete_sequence=['steelblue']
    )

    min_wv = np.floor(data_handler.wavelengths.min() / 100) * 100
    max_wv = np.ceil(data_handler.wavelengths.max() / 100) * 100
    tick_values = np.arange(min_wv, max_wv + 100, 100)

    fig.update_layout(
        width=550,
        height=360,
        title={
            'text': "Spectrum",
            'x': 0.5,
            'y': 0.9,
            'xanchor': 'center',
            'font': {'size': 22}
        },
        margin=dict(l=70, r=70, t=70, b=70),
        xaxis=dict(
            title={
                'text': "Wavelengths (nm)",
                'font': {'size': 20}
            },
            tickfont={'size': 16, 'color': 'black'},
            showline=True,
            linewidth=2,
            linecolor='black',
            mirror=False,
            showgrid=False,
            zeroline=False,
            tickmode='array',
            tickvals=tick_values
        ),
        yaxis=dict(
            title={
                'text': "Intensity (arb.un.)",
                'font': {'size': 20}
            },
            tickfont={'size': 16, 'color': 'black'},
            showline=True,
            linewidth=2,
            linecolor='black',
            mirror=False,
            showgrid=False,
            zeroline=False,
            tickmode='array',
            tickvals=tick_values
        ),
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        font=dict(color='black')
    )
    return fig


create_average_spectrum_plot(data_handler)

In [6]:
counts = analytics.identify_from_elements(spectral_cube = data_handler.dataset,
                                            wavelengths = data_handler.wavelengths,
                                            operation = 'average',
                                            min_intensity = 0.1,
                                            return_counts = True)

## PERIODIC TABLE ###
fig2 = visual.dash_periodic_table(counts)
fig2.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
)

In [211]:
def create_labels_image_plot(data_handler):
    slice_data = data_handler
    
    # Create the figure without colorbar and title
    fig = px.imshow(
        slice_data,
        color_continuous_scale='turbo',
        labels={'color': 'Class'}  # Label for the intensity values
    )

    # Remove the colorbar and title
    fig.update_layout(
        width = 340,
        height = 340,
        title = {
            'text': f"Final Classification", 
            'x': 0.53,
            'xanchor': 'center',
            'y': 0.95,
            'font': {'size': 22}
        },
        margin=dict(l=5, r=5, t=5, b=5),
        xaxis=dict(
            title={
                'text': "x (mm)",
                'font': {'size': 18},
                'standoff': 0,
            },
            range=[0, slice_data.shape[1] - 1],
            tickvals=[1, slice_data.shape[1] - 1],
            ticktext=[0, slice_data.shape[1]],
            tickfont=dict(size=16, color='black'),
            linewidth=2,
            linecolor='black',
            mirror=True
        ),
        yaxis=dict(
            title={
                'text': "y (mm)",
                'font': {'size': 18},
                'standoff': 0,
            },
            range=[0, slice_data.shape[0] - 1],
            tickvals=[1, slice_data.shape[0] - 1],
            ticktext=[slice_data.shape[0], 0],
            tickfont=dict(size=16, color='black'),
            linewidth=2,
            linecolor='black',
            mirror=True
        ),
        plot_bgcolor='rgba(0,0,0,0)',  # Transparent background
        paper_bgcolor='rgba(0,0,0,0)',  # Transparent background
        font=dict(color='black'),
        coloraxis_showscale=False  # This removes the colorbar
    )
    return fig

create_labels_image_plot(labels)

In [308]:
def create_periodic_table_plot(data_handler):
    counts = analytics.identify_from_elements(
        spectral_cube=data_handler.dataset,
        wavelengths=data_handler.wavelengths,
        operation='average',
        min_intensity=0.1,
        return_counts=True
    )
    
    fig = visual.dash_periodic_table(counts, figsize = (900, 350))
    fig.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        margin=dict(l=25, r=25, t=25, b=25),
    )
    fig.update_xaxes(range=[0, 19])  # Reduce the horizontal space by tightening the range around the elements
    fig.update_yaxes(scaleanchor="x", scaleratio=1) 

    return fig

create_periodic_table_plot(data_handler)

In [309]:
from dash import Dash, html, dcc
import dash_bootstrap_components as dbc

# Initialize the app
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Constants
SAMPLE_ID = "[sample-id]"
PROJECT_ID = "[project-id]"

def create_title_section():
    return html.Div([
        html.H1("Spectral UI"),
        html.P([
            html.Br(),
            html.Span(f"SAMPLE: {SAMPLE_ID}"),
            html.Br(),
            html.Br(),
            html.Span(f"This interface presents a resumed analysis of the finding for the spectral data collected, as part of PROJECT {PROJECT_ID}."),
            html.Br(),
            html.Br(),
            html.Span("by INESC TEC Spectroscopy Group"),
        ])
    ], className="title-section", style=None)

def create_radio_section():
    return html.Div([
        dbc.RadioItems(
            options=[
                {"label": "Analysis", "value": 1},
                {"label": "Report", "value": 2},
                {"label": "About", "value": 3},
            ],
            value=1,
            className="btn-group",
            inputClassName="btn-check",
            labelClassName="btn btn-outline-light",
            labelCheckedClassName="btn btn-light",
        )
    ], className="radio-section", style=None)

def create_plot_section1(fig):
    return html.Div([
        dcc.Graph(
            id='imshow-plot',
            figure=fig,
            config={'displayModeBar': False}
        )
    ], className="plot-section1", style=None)

def create_plot_section2(fig1, fig):
    return html.Div([
        html.Div(
            dcc.Graph(
                id='spectrum-plot',
                figure=fig1,
                config={'displayModeBar': False}
            ), 
            style=None,
            className='section2-plot' 
        ),
        html.Div(
            dcc.Graph(
                id='spectrum-imshow',
                figure=fig,
                config={'displayModeBar': False}
            ),  
            style=None,
            className='section2-imshow' 
        )
    ], className="plot-section2")

def create_plot_section3(fig2):
    return html.Div([
        dcc.Graph(
            id='periodic-table',
            figure=fig2,
            config={'displayModeBar': False}
        )
    ], className="plot-section3", style=None)

def create_left_panel(result_image_fig):
    return html.Div([
        create_title_section(),
        create_radio_section(),
        create_plot_section1(result_image_fig)
    ], className="left-panel", style=None)

def create_right_panel(average_spectrum_fig, spectral_image_fig, periodic_table_fig):
    return html.Div([
        create_plot_section2(average_spectrum_fig, spectral_image_fig),
        create_plot_section3(periodic_table_fig)
    ], className="right-panel", style=None)

# Create plots
periodic_table_fig = create_periodic_table_plot(data_handler)
spectral_image_fig = create_spectral_image_plot(data_handler)
average_spectrum_fig = create_average_spectrum_plot(data_handler)
classification_fig = create_labels_image_plot(labels)

# Set up the layout
app.layout = dbc.Container([
    html.Div([
        create_left_panel(classification_fig),
        create_right_panel(average_spectrum_fig, spectral_image_fig, periodic_table_fig)
    ], className="main-container", style=None)
], 
fluid=True,
className='dashboard-container')

# Main execution
if __name__ == '__main__':
    app.run_server(debug=True)

# Interface Only

In [181]:
from dash import Dash, html, dcc
import dash_bootstrap_components as dbc

# Initialize the app
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Styles
None = {
    'outline': '2px solid #333',
    'border-radius': '4px',
    'padding': '0px',
    'margin': '0px'
}

# Constants
SAMPLE_ID = "[sample-id]"
PROJECT_ID = "[project-id]"

# Components
def create_title_section():
    return html.Div([
        html.H1("Spectral UI"),
        html.P([
            html.Br(),
            html.Span(f"SAMPLE: {SAMPLE_ID}"),
            html.Br(),
            html.Br(),
            html.Span(f"This interface presents a resumed analysis of the finding for the spectral data collected, as part of PROJECT {PROJECT_ID}."),
            html.Br(),
            html.Br(),
            html.Span("by INESC TEC Spectroscopy Group"),
        ])
    ], className="title-section", style=None)

def create_radio_section():
    return html.Div([
        dbc.RadioItems(
            options=[
                {"label": "Analysis", "value": 1},
                {"label": "Report", "value": 2},
                {"label": "About", "value": 3},
            ],
            value=1,
            className="btn-group",
            inputClassName="btn-check",
            labelClassName="btn btn-outline-light",
            labelCheckedClassName="btn btn-light",
        )
    ], className="radio-section", style=None)

def create_plot_section1():
    return html.Div([
        dcc.Graph(
            id='imshow-plot',
            figure=fig,  # Note: 'fig' needs to be defined
            config={'displayModeBar': False}
        )
    ], className="plot-section1", style=None)

def create_plot_section2():
    return html.Div([
        html.Div(
            dcc.Graph(
                id='spectrum-plot',
                figure=fig1,  # Note: 'fig1' needs to be defined
                config={'displayModeBar': False}
            ), 
            style=None,
            className='section2-plot' 
        ),
        html.Div(
            dcc.Graph(
                id='spectrum-imshow',
                figure=fig,  # Note: 'fig' needs to be defined
                config={'displayModeBar': False}
            ),  
            style=None,
            className='section2-imshow' 
        )
    ], className="plot-section2")

def create_plot_section3():
    return html.Div([
        dcc.Graph(
            id='periodic-table',
            figure=fig2,  # Note: 'fig2' needs to be defined
            config={'displayModeBar': False}
        )
    ], className="plot-section3", style=None)

def create_left_panel():
    return html.Div([
        create_title_section(),
        create_radio_section(),
        create_plot_section1()
    ], className="left-panel", style=None)

def create_right_panel():
    return html.Div([
        create_plot_section2(),
        create_plot_section3()
    ], className="right-panel", style=None)

# Layout assembly
app.layout = dbc.Container([
    html.Div([
        create_left_panel(),
        create_right_panel()
    ], className="main-container", style=None)
], 
fluid=True,
className='dashboard-container')

if __name__ == '__main__':
    app.run_server(debug=True)

SyntaxError: cannot assign to None (1200983966.py, line 8)